<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/transformers_asr_pipeline_optimisations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A walkthrough of the pleathora of optimisations and nifty features offered via `pipeline` for Automatic Speech Recognition! 🤗

from yours truly - [Vaibhav (VB) Srivastav](https://twitter.com/reach_vb) 🤙

In [1]:
!pip install -q transformers accelerate datasets optimum
!pip install -q ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.1/396.1 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%load_ext autotime

time: 384 µs (started: 2023-09-13 11:42:58 +00:00)


In [3]:
import torch
from transformers import pipeline

time: 16.7 s (started: 2023-09-13 11:43:01 +00:00)


In [4]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

time: 1min 26s (started: 2023-09-13 10:15:34 +00:00)


## Download a test file!

This is a 2 hour long audio snippet from a revdotcoms Earnings22 dataset.

In [6]:
!wget https://github.com/revdotcom/speech-datasets/raw/main/earnings22/media/4469669.mp3

--2023-09-13 11:46:32--  https://github.com/revdotcom/speech-datasets/raw/main/earnings22/media/4469669.mp3
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/revdotcom/speech-datasets/main/earnings22/media/4469669.mp3 [following]
--2023-09-13 11:46:32--  https://media.githubusercontent.com/media/revdotcom/speech-datasets/main/earnings22/media/4469669.mp3
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29628186 (28M) [audio/mpeg]
Saving to: ‘4469669.mp3’

4469669.mp3         100%[===================>]  28.25M   149MB/s    in 0.2s    

2023-09-13 11:46:33 (149 MB/s) - ‘4469669.mp3’ s

## Inference

By default Whisper is only capable of running inference for 30 second audios max.

In [6]:
pipe("4469669.mp3")

{'text': " Now it's time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session on an online basis. From 4 to 5 o'clock we will be presenting from our side and followed by a 30-minute question session for the media. The questions from analysts"}

time: 28.3 s (started: 2023-09-13 10:17:24 +00:00)


# Chunking

In [7]:
pipe("4469669.mp3", chunk_length_s=20)["text"][:200]

" Now it's time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session o"

time: 25min 2s (started: 2023-09-13 10:19:49 +00:00)


It took about 25 minutes and 2 seconds to run inference on a 2 hour long snippet.

Can we make this faster?

## Cue.. Batching

In [8]:
pipe("4469669.mp3",
     chunk_length_s=20,
     batch_size=8)["text"][:200]

" Now it's time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session o"

time: 13min 36s (started: 2023-09-13 10:45:21 +00:00)


Voila!! It only took the `pipeline` 13 minutes and 36 seconds to run inference this time. We were able to reduce the inference time by 50%. 🚀

But, the real question is - can we make it even faster? 💨

## Half - Precision

In [10]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

time: 1min 30s (started: 2023-09-13 11:01:04 +00:00)


In [11]:
pipe("4469669.mp3",
     chunk_length_s=20,
     batch_size=8)["text"][:200]

" Now it's time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session o"

time: 6min 38s (started: 2023-09-13 11:03:11 +00:00)


## BetterTransformer

In [4]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

pipe.model = pipe.model.to_bettertransformer()

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


time: 2min 19s (started: 2023-09-13 11:43:43 +00:00)


In [7]:
pipe("4469669.mp3",
     chunk_length_s=20,
     batch_size=8)["text"][:200]

" Now it's time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session o"

time: 5min 35s (started: 2023-09-13 11:47:26 +00:00)
